In [82]:
import os
import pandas as pd
import numpy as np
import glob
from itertools import cycle
import matplotlib.pyplot as plt
from scipy import stats
import shutil

In [107]:
path = str(glob.os.getcwd())

user = path.split('\\')[2]

folder = "Lab_Sheets/"

datasheets = '/Users/' + user + '/Desktop/Coding/StandardLab/' + folder

glob.os.chdir(datasheets)

for x, folder, file in os.walk(datasheets):
    
    for loc in folder[0:2]:

        folder_loc = datasheets + loc
        
        glob.os.chdir(folder_loc)
        
        all_datasheets = glob.glob(folder_loc + "/*.csv")
        
        N_Re_Run = []
        U_Re_Run = []
        
        for sheet in all_datasheets:
            lab_data = pd.read_csv(sheet)
    
            sheet_name = sheet.split('\\')[-1]
            title = sheet_name.split('_')[2]

            # Checking for data - skipping remainder of code (breaking loop) if there is no data

            if np.isnan(lab_data.Absorbance[0]):
                break

            # Continuing if data is found

            else:
                print(sheet)


                # Identifying standard absorbances based on 'Type' column, and adding to list of values to plot
                
                curve_absorbance = []
                curve_concentration = []

                for n in range(len(lab_data)):
                    if lab_data.Type[n] == 'C':
                        curve_absorbance.append(int(lab_data.Absorbance[n]))
                        curve_concentration.append(int(lab_data.Sample_ID[n]))


            # calculating r^2 value for two lists of values

            slope, intercept, r_value, p_value, std_err = stats.linregress(curve_absorbance, curve_concentration)
            print(r_value**2)

            # creating conditional for screening based on r^2 value
            if r_value**2 < 0.8:

                # Moving bad curve sheet to new folder to avoid running again in the future
                sheet_name = sheet.split('\\')[-1]
                shutil.move(sheet, '/Users/' + user + '/Desktop/Coding/StandardLab/Questionable_Data/BADCURVE_' + str(sheet_name))

                # Creating new blank dataframe for running samples again

                sheet_redo = lab_data
                sheet_redo['Sample_Wt(g)'] = ''
                sheet_redo['Absorbance'] = ''

                # Reassigning navigation variables to save ouput files in a new location

                folder = "Lab_Sheets/"

                save_loc = '/Users/' + user + '/Desktop/Coding/StandardLab/' + folder + str(loc)


                # Navigating to specified saving location

                glob.os.chdir(save_loc)


                # Saving new dataframe in Lab_Sheets folder with 'B' to indicate re-running samples

                sheet
                sheet_redo.to_csv('B_' + '_'.join(sheet_name.split('_')[1:]), index=False)


            # if the r^2 value is acceptable, look at variation between samples...

            else: 
                # define which rows contain data for samples, and creating new df with only absorbance
        
                sample_data = lab_data[lab_data.Type != 'C']
                sample_data = sample_data[sample_data.Type != 'B']
                sample_data_ab = sample_data[['Sample_ID', 'Absorbance']]


                # group sample data based on Sample_ID information

                sample_data_g = sample_data_ab.groupby('Sample_ID')


                # merge and create new dataframe for variance values

                sample_data_var = sample_data_ab.merge((abs((sample_data_g.max() - sample_data_g.min())) / (sample_data_g.max())), on='Sample_ID')


                # create conditional based on variance column

                for i, row in sample_data_var.iterrows():
                    if float(row['Absorbance_y']) > 0.1:

                        # Adding sample id information to list if condition is met
                        if title == "URE":
                            U_Re_Run.append(row['Sample_ID'])
                        elif title == "NIT":
                            N_Re_Run.append(row['Sample_ID'])
                        

        # Adding list of ID's to fill column in dataframe

        title1 = sheet_name.split('_')[3]
        title2 = sheet_name.split('_')[4]
        titles = ('2', title1, title2)
        file_name = ('_').join(titles)
        file_name  

        # Creating a new dataframe with sample ID's to be run again

        if U_Re_Run != []:

            USamples_Re_Run = lab_data
            USamples_Re_Run['Sample_Wt(g)'] = ''
            USamples_Re_Run['Absorbance'] = ''
            USamples_Re_Run['Type'] = ''
            USamples_Re_Run['Sample_ID'] = ''

            ID_List = ['1_1', '1_2', '2_1', '2_2', 0.00, 0.00, 0.00, 549.5825955, 549.5825955, 549.5825955, 1099.165191, 1099.165191, 1099.165191, 4396.660764, 4396.660764, 4396.660764]

            USamples_Re_Run.Sample_ID[0:16] = ID_List
            USamples_Re_Run.Sample_ID[16:(16+len(U_Re_Run))] = U_Re_Run

            seq = cycle(['O', 'D'])

            type_list = ['B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']

            USamples_Re_Run['Type'] = [next(seq) for i in range(len(USamples_Re_Run))]
            USamples_Re_Run.Type[0:16] = type_list
            USamples_Re_Run.Type[(16+len(U_Re_Run)):] = ''

            glob.os.chdir(datasheets + '/' + str(loc))
    
            USamples_Re_Run.to_csv('URE_' + file_name + '.csv', index=False)
            print("Questionable variance found in one or more URE samples. A list of these ID's has been created and saved in the Lab Sheets folder as " + file_name + '.csv.')

        if N_Re_Run != []:

            NSamples_Re_Run = lab_data
            NSamples_Re_Run['Sample_Wt(g)'] = ''
            NSamples_Re_Run['Absorbance'] = ''
            NSamples_Re_Run['Type'] = ''
            NSamples_Re_Run['Sample_ID'] = ''

            ID_List = ['1_1', '1_2', '2_1', '2_2', 0.00, 0.00, 0.00, 1500.00, 1500.00, 1500.00, 3000.00, 3000.00, 3000.00, 4500.00, 4500.00, 4500.00]

            NSamples_Re_Run.Sample_ID[0:16] = ID_List
            NSamples_Re_Run.Sample_ID[16:(16+len(N_Re_Run))] = N_Re_Run

            seq = cycle(['O', 'D'])

            type_list = ['B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']

            NSamples_Re_Run['Type'] = [next(seq) for i in range(len(NSamples_Re_Run))]
            NSamples_Re_Run.Type[0:16] = type_list
            NSamples_Re_Run.Type[(16+len(N_Re_Run)):] = ''

            glob.os.chdir(datasheets + '/' + str(loc))
            
            NSamples_Re_Run.to_csv('NIT_' + file_name + '.csv', index=False)
            print("Questionable variance found in one or more NIT samples. A list of these ID's has been created and saved in the Lab Sheets folder as " + file_name + '.csv.')

/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/MR_USB\A_1_NIT_MR_USB_00_00_00 (good curve).csv
1.0
OOPS
Questionable variance found in one or more NIT samples. A list of these ID's has been created and saved in the Lab Sheets folder as 2_MR_USB.csv.
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/RV_CTS\A_1_NIT_RV_CTS_00_00_00 (bad curve).csv
0.7211581696030183
OOPS
HI


In [106]:
path = str(glob.os.getcwd())

user = path.split('\\')[2]

folder = "Lab_Sheets/"

datasheets = '/Users/' + user + '/Desktop/Coding/StandardLab/' + folder

glob.os.chdir(datasheets)

for x, folder, file in os.walk(datasheets):
    
    for loc in folder:
        print(loc)

MR_USB
RV_CTS


In [159]:
from string import ascii_uppercase


LETTERS = {i: str(letter) for i, letter in enumerate(ascii_uppercase, start=1)} 

ascii_uppercase.index('B')

# ascii_uppercase.rindex()

TypeError: rindex() takes at least 1 argument (0 given)